In [13]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [14]:
filename = "../../data/03/ABox.ttl"

In [15]:
text1 = '''CQ_3.1
What are the probable estimates of the A-score, B-score, and C-score for each risk affecting each heritage asset, and the documents and textual notes recording them?
'''

query1 = '''
PREFIX : <https://w3id.org/sirius/ontology/data/03/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX hero: <https://w3id.org/sirius/ontology/hero/>

SELECT ?risk ?heritage_asset ?risk_component ?probable_estimate ?note ?knowledge_source
WHERE {
  ?risk_assessment hero:describes ?heritage_asset ;
                    hero:analyses ?risk ;
                    hero:quantifies ?risk_component . 
  ?risk_component a ?component_class ;
                  hero:hasProbableEstimate ?probable_estimate ;
                  hero:hasNote ?note .
  OPTIONAL {
    ?risk_component hero:isDocumentedBy ?knowledge_source ;
  }
  FILTER (
    ?component_class = hero:Frequency ||
    ?component_class = hero:FractionalValueLoss ||
    ?component_class = hero:Exposure 
  )
}
'''

In [16]:
text2 = '''CQ_3.2
What are the low, probable, and high estimates of the magnitudes of risk for each risk associated with each heritage asset?
'''

query2 = '''
PREFIX : <https://w3id.org/sirius/ontology/data/03/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX hero: <https://w3id.org/sirius/ontology/hero/>

SELECT ?risk ?heritage_asset ?low_estimate ?probable_estimate ?high_estimate
WHERE {
  ?risk_assessment hero:describes ?heritage_asset ;
                    hero:analyses ?risk ;
                    hero:quantifies ?risk_magnitude .
  ?risk_magnitude a hero:Magnitude ;
                  hero:hasLowEstimate ?low_estimate ;
                  hero:hasProbableEstimate ?probable_estimate ;
                  hero:hasHighEstimate ?high_estimate .
}
'''

In [17]:
text3 = '''CQ_3.3
What are the low, probable, and high estimates of the A-score, B-score, C-score, and magnitude of risk for each risk affecting each heritage asset?
'''

query3 = '''
PREFIX hero: <https://w3id.org/sirius/ontology/hero/>

SELECT ?risk ?component_class ?low_estimate ?probable_estimate ?high_estimate
WHERE {
  ?risk_assessment hero:describes ?heritage_asset ;
                    hero:analyses ?risk ;
                    hero:quantifies ?risk_component .
  ?risk_component a ?component_class ;
                  hero:hasLowEstimate ?low_estimate ;
                  hero:hasHighEstimate ?high_estimate ;
                  hero:hasProbableEstimate ?probable_estimate .
  FILTER (
    ?component_class = hero:Frequency ||
    ?component_class = hero:FractionalValueLoss ||
    ?component_class = hero:Exposure ||
    ?component_class = hero:Magnitude
  )
}
'''

In [18]:
queries = [(text1, query1),
           (text2, query2),
           (text3, query3)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_3.1
What are the probable estimates of the A-score, B-score, and C-score for each risk affecting each heritage asset, and the documents and textual notes recording them?

+----------------------------------------------------------------+--------------------------------------------------+------------------------------------------------------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------+
| risk                                                           | heritage_asset                                   | risk_component                                                   |   probable_estimate | note                                                                            